In [145]:
import pandas as pd
import numpy as np
imp

In [146]:
df_true = pd.read_csv('True.csv')

In [147]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [148]:
df_true.shape

(21417, 4)

In [149]:
df_fake = pd.read_csv('Fake.csv')

In [150]:
df_fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [151]:
df_fake.shape

(23481, 4)

# Data Cleaning

In [152]:
df_true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [153]:
df_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


Since we will be basing our model on text data, out column are not relevant to our model; therefor all other columns will be drop except "text".

In [154]:
df_fake =  df_fake.drop_duplicates() #dropping duplicated values

In [155]:
df_fake = df_fake.dropna() #dropping nan values

In [156]:
df_true = df_true.dropna() #dropping nan values

In [157]:
df_fake.info() #checking to see if nan values are dropped 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23478 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23478 non-null  object
 1   text     23478 non-null  object
 2   subject  23478 non-null  object
 3   date     23478 non-null  object
dtypes: object(4)
memory usage: 917.1+ KB


In [158]:
df_true.info() #checking to see if nan values are dropped

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [159]:
df_fake = df_fake.drop(columns=['title', 'subject', 'date']) #dropping unused variables

In [160]:
df_fake.sample(5) #sampling the dataset

,text
6544,There has been a national uproar over HB2 ever...
1325,Michael Cohen is a man who knows Donald Trump ...
23191,Tune in to the Alternate Current Radio Network...
12162,"After the Reid event, Hillary Clinton greeted ..."
15296,The Obama family continues to rack up huge bil...


In [161]:
df_true = df_true.drop(columns=['title', 'date','subject'])

In [162]:
df_true.sample()

,text
313,FRANKFURT/WASHINGTON (Reuters) - A U.S. feder...


# EDA

We will add a new column that will either be 0 or 1 based on if the news is fake or real. Fake news will be given 0 and real news will be given 1. 

In [163]:
df_fake['Target'] = 0 #adding target column to the dataset

In [164]:
df_fake.sample(5) #sampling the data

,text,Target
12179,Everybody in the building is sick of it. We a...,0
14452,"Since this article was written in 2006, the il...",0
12975,How many Hillary supporters does it take to fi...,0
8693,There are seldom those moments in life that se...,0
19395,MASSIVE PLANNED MALL FIGHTS Broke out across t...,0


In [165]:
df_true['Target'] = 1 #adding target column to the data

In [166]:
df_true.sample(5) #sample of the dataset aftering adding "Target" column

,text,Target
15090,"DANANG, Vietnam (Reuters) - U.S. President Don...",1
15360,GENEVA (Reuters) - North Korea said on Wednesd...,1
20537,"BEIRUT/AL SHADADI, Syria (Reuters) - U.S.-back...",1
19367,WASHINGTON (Reuters) - Expanded negotiations a...,1
18474,WASHINGTON (Reuters) - The U.S. Supreme Court ...,1


Now we will concat both dataset into metadata

In [167]:
data = pd.concat([df_fake, df_true]) #concat the two dataset

In [168]:
data.sample(5) #sample of the dataset

,text,Target
2627,WASHINGTON (Reuters) - President Donald Trump ...,1
17165,(This version of the October 15 story correct...,1
3849,The effects of a Donald Trump presidency are a...,0
11315,PARIS (Reuters) - President Emmanuel Macron ha...,1
3570,Donald Trump just got taken to the woodshed fo...,0


In [169]:
data.shape

(44895, 2)